#### Test get pictures from Tvåkammar-riksdagen 1867

* this [Notebook](https://github.com/salgo60/Wikidata_riksdagen-corpus/blob/main/Notebook/Sveriges%20Riksdag%20i%20SPA%20-%20Tvåkammar-riksdagen%201867.ipynb)
  * see also 
      * [Notebook Sveriges Riksdag SPA](https://github.com/salgo60/Wikidata_riksdagen-corpus/blob/main/Notebook/Sveriges%20Riksdag%20i%20SPA.ipynb)
      

See GITHUB discussion about different nameforms (WIkidata alias) [welfare-state-analytics/riksdagen-corpus/issues/103#issuecomment-999775874](https://github.com/welfare-state-analytics/riksdagen-corpus/issues/103#issuecomment-999775874) 

The following Swedish PM books has been found at https://portrattarkiv.se/ that could help us get more pictures of Swedish PM --> and also name forms as 

| PortraitCatalog | Number records | uploaded y/n 
| :------------ | -------: | -------- |
|[Sveriges Riksdag 1924](https://portrattarkiv.se/advancedsearch;facts=PortraitCatalog:Sveriges%20Riksdag%201924) | 382 records | n
|[Tvåkammar-riksdagen 1867-](https://portrattarkiv.se/advancedsearch;facts=PortraitCatalog:Tv%C3%A5kammar-riksdagen%201867-)|4351 records | n



 
### API for a record
eg. [rkIpfGqHXrAAAAAAAACvFw](https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAACvFw) --> API get https://portrattarkiv.se/endpoints/portraits.php?id=rkIpfGqHXrAAAAAAAACvFw 

--> you can see 
>"Id": "640330", 
>"Category": "Sysselsättning", 
>"Type": "Authority", 
>"Value": "Riksdagens andra kammare", 
>"Modified": "2018-03-01 12:01:55", 



In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-01-03 18:15:30.907929


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/search.php"  
urlbasePic = "https://portrattarkiv.se/endpoints/file.php?id="

In [3]:
PortraitCatalogSPA = "Tvåkammar-riksdagen 1867-"
encoded_body = json.dumps({
        "limit": "10000",
        "from": "0",
        "facts":{},
        "factsExact":{"PortraitCatalog":PortraitCatalogSPA}
    })

In [4]:
r = http.request('POST', url,
             headers={'Content-Type': 'application/json'},
             body=encoded_body)

if r.status != 200:
    print(r.status)
    

data = json.loads(r.data.decode('utf-8'),) 
urls = []

print("Number hits \t",len(data["hits"]["hits"]))


Number hits 	 4351


In [5]:
header = ['id', 'urlPicture', 'FirstName','LastName','BirthYear','PortraitCatalog']
list_pm = []

for h in data["hits"]["hits"]:
    id = h["_id"]
    source = h["_source"]
    #print(id)
    try:
        urlPicture = urlbasePic + id
        #print("\t",urlPicture)
        urls.append(urlPicture)
        score = h["_score"]
        FirstName = source["FirstName"]
        LastName = source["LastName"]
        BirthYear = source["BirthYear"]
        BirthDate = source["BirthDate"]
        #print(id, FirstName, " ", LastName, " - ", BirthYear )
        list_pm.append([id,urlPicture,FirstName,LastName,BirthDate,PortraitCatalogSPA])       
    except:
        print("Error hdata")
        pass

In [6]:
#list_pm 
list_pm_df = pd.DataFrame(list_pm)
list_pm_df.columns = header

In [7]:
list_pm_df.shape

(4351, 6)

In [8]:
from IPython.display import HTML, Image

def _src_from_data(data):
    """Base64 encodes image bytes for inclusion in an HTML img element"""
    img_obj = Image(data=data)
    for bundle in img_obj._repr_mimebundle_():
        for mimetype, b64value in bundle.items():
            if mimetype.startswith('image/'):
                return f'data:{mimetype};base64,{b64value}'

def gallery(images, row_height='auto'):
    """Shows a set of images in a gallery that flexes with the width of the notebook.
    
    Parameters
    ----------
    images: list of str or bytes
        URLs or bytes of images to display

    row_height: str
        CSS height value to assign to all images. Set to 'auto' by default to show images
        with their native dimensions. Set to a value like '250px' to make all rows
        in the gallery equal height.
    """
    figures = []
    for image in images:
        if isinstance(image, bytes):
            src = _src_from_data(image)
            caption = ''
        else:
            src = image
            caption = f'<figcaption style="font-size: 0.6em">{image}</figcaption>'
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="{src}" style="height: {row_height}">
              {caption}
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

In [9]:
urls = []
for h in data["hits"]["hits"][1:10]:
    id = h["_id"]
    source = h["_source"]
    urlPicture = urlbasePic + id
    urls.append(urlPicture)
gallery(urls, row_height='100px')

In [10]:
csvFile = "OpenRefine_" + PortraitCatalogSPA + ".csv"
print(csvFile)
list_pm_df.to_csv(csvFile)

OpenRefine_Tvåkammar-riksdagen 1867-.csv


In [11]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-01-03 18:15:44.701850
Time elapsed (hh:mm:ss.ms) 0:00:13.794501
